In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 텐서로 변환 (픽셀 값을 [0,1]로 정규화)
    transforms.Normalize((0.1307,), (0.3081,))  # 평균과 표준편차로 정규화
])


# 데이터셋 로드
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# 데이터 로더 설정
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# LSTM 모델 정의
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers): # input_size: 입력 특성의 크기 (여기서는 이미지의 각 픽셀 행의 길이인 28), hidden_size: LSTM의 은닉 상태 크기, output_size: 출력 클래스 수 (MNIST는 0~9까지 10개 클래스), num_layers: LSTM 층의 수
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM 층 정의
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # 최종 출력층 정의
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 은닉 상태와 셀 상태를 0으로 초기화
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # (층 수, 배치 크기, 은닉 상태 크기)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # LSTM에 입력
        out, _ = self.lstm(x, (h0, c0))  # out: (배치 크기, 시퀀스 길이, 은닉 상태 크기)

        # 최종 타임스텝의 은닉 상태를 사용하여 출력 계산
        out = self.fc(out[:, -1, :])  # out[:, -1, :] => (배치 크기, 은닉 상태 크기)

        return out

# MNIST 이미지는 (배치 크기, 1, 28, 28) 형태의 4차원 텐서로 로드
# LSTM에 입력하려면 (배치 크기, 시퀀스 길이, 입력 크기) 형태의 3차원 텐서로 변환

input_size = 28       # 각 타임스텝당 입력 크기 (이미지의 열 개수)
hidden_size = 128     # 은닉 상태 크기
output_size = 10      # 출력 클래스 수
num_layers = 2        # LSTM 층 수


model = LSTMNet(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # 모델을 학습 모드로 전환
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # 입력 데이터 형태 변환
        data = data.view(-1, 28, 28)  # (배치 크기, 시퀀스 길이, 입력 크기)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


# 평가 함수
def test(model, device, test_loader):
    model.eval()  # 모델을 평가 모드로 전환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # 입력 데이터 형태 변환
            data = data.view(-1, 28, 28)

            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)  # 배치 손실 합산
            pred = output.argmax(dim=1, keepdim=True)  # 가장 높은 값을 가진 클래스 선택
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
        f' ({accuracy:.2f}%)\n')


# 학습 및 평가 실행
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295057
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.676799
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.178936
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.184467
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.129379
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.154680
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.156757
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.123943
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.102413
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.115079

Test set: Average loss: 0.1015, Accuracy: 9709/10000 (97.09%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.072166
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.155104
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.065627
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.008433
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.032366
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.028768
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.016187
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.184977
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.07022